## Subgraph Classification with Average Pooling - Variable Misuse

The pooling function is made of average Pooling

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch[10:53:36] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: libtorch_cuda_cpp.so: cannot open shared object file: No such file or directory



# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "10_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [4]:
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Correct            35036
Variable misuse    35036
Name: label, dtype: int64

dev
Correct            3754
Variable misuse    3754
Name: label, dtype: int64

eval
Variable misuse    18787
Correct            18787
Name: label, dtype: int64



# Config

In [5]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=0,
    epochs=10, 
    
    train_frac=0.8,
    random_seed=42, 
    
    # model parameters
    elem_emb_size=300,
    node_emb_size=300,                  # *** dimensionality of node embeddings
    h_dim=300,                           # *** should match to node dimensionality
    n_layers=5,
    dropout=0.1,
    activation="tanh"
)

In [6]:
save_config(config, "average_pooling_var_misuse_subgraph.yaml")

# Create Dataset

In [7]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [8]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [9]:
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective

class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

## Launch Tensorboard

In [10]:
# %tensorboard --logdir './10_percent_v1'

## Start model training and Evaluation

In [11]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Number of nodes 4900346


2022-07-26 11:06:38.195999: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Epoch 1:   0%|                                          | 0/548 [00:00<?, ?it/s]

Epoch: 0, Time: 1025 s
{'Accuracy/test/VariableMisuseClf_': 0.6478164480077746,
 'Accuracy/train/VariableMisuseClf_': 0.5714285714285714,
 'Accuracy/train_avg/VariableMisuseClf': 0.5422009417361835,
 'Accuracy/val/VariableMisuseClf_': 0.6484627219531881,
 'Loss/test/VariableMisuseClf_': 0.6192039107384325,
 'Loss/train/VariableMisuseClf_': 0.6782256364822388,
 'Loss/train_avg/VariableMisuseClf': 0.6808068071403642,
 'Loss/val/VariableMisuseClf_': 0.6174985321901613,
 'grad_norm/train/_': 0.6049430126716081}


Epoch 1: 100%|████████████████████████████████| 548/548 [10:09<00:00,  1.11s/it]


Epoch: 1, Time: 964 s
{'Accuracy/test/VariableMisuseClf_': 0.7025388726919339,
 'Accuracy/train/VariableMisuseClf_': 0.6785714285714286,
 'Accuracy/train_avg/VariableMisuseClf': 0.6717874250521376,
 'Accuracy/val/VariableMisuseClf_': 0.7020530669895076,
 'Loss/test/VariableMisuseClf_': 0.5737367088900132,
 'Loss/train/VariableMisuseClf_': 0.6342061161994934,
 'Loss/train_avg/VariableMisuseClf': 0.5924317710591058,
 'Loss/val/VariableMisuseClf_': 0.5730623188665358,
 'grad_norm/train/_': 0.4858681494617203}
Final eval: Loss/val/VariableMisuseClf_final: 0.5730623188665358, Accuracy/val/VariableMisuseClf_final: 0.7020530669895076, Loss/test/VariableMisuseClf_final: 0.5737367088900132, Accuracy/test/VariableMisuseClf_final: 0.7025388726919339


(<__main__.Trainer at 0x7f434b144070>,
 {'Loss/val/VariableMisuseClf_final': 0.5730623188665358,
  'Accuracy/val/VariableMisuseClf_final': 0.7020530669895076,
  'Loss/test/VariableMisuseClf_final': 0.5737367088900132,
  'Accuracy/test/VariableMisuseClf_final': 0.7025388726919339})

In [12]:
!nvidia-smi

Tue Jul 26 11:45:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   37C    P0    72W / 300W |  17948MiB / 32510MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------